# Data Mining Project
---

Import βιβλιοθήκες

In [2]:
import pandas as pd
import numpy as np

In [ ]:
# df_raw = pd.read_csv('household_power_consumption.txt', sep=';', low_memory=False)
#
# day_data = df_raw[df_raw['Date'] == '28/4/2007']
#
# print("Sample of missing values on 28/04/2007: ")
# display(day_data.iloc[15:30])

In [3]:
df = pd.read_csv('household_power_consumption.txt', sep=';', na_values=['?'], low_memory=False)

print("\nNumber of missing values before cleaning")
print(df.isna().sum())


Number of missing values before cleaning
Date                         0
Time                         0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


Συμπλήρωση ελλιπών τιμών με ffill()

In [4]:
df.ffill(inplace=True)

In [5]:
print("\nNumber of missing values after cleaning")
print(df.isna().sum())


Number of missing values after cleaning
Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64


Μετατροπή σε Datetime

In [6]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
df.set_index('datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)

# Δημιουργία του Sub_metering_4 (Λοιπές συσκευές)
# Τύπος ακριβώς όπως ορίζεται στα Notes:
# (global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3)

df['Sub_metering_4'] = (df['Global_active_power'] * 1000 / 60) - df['Sub_metering_1'] - df['Sub_metering_2'] - df['Sub_metering_3']

# Καθαρισμός τυχόν αρνητικών τιμών που μπορεί να προκύψουν από μικρο-αποκλίσεις μετρήσεων
df['Sub_metering_4'] = df['Sub_metering_4'].apply(lambda x: 0 if x < 0 else x)

print("Το χαρακτηριστικό 'Sub_metering_4' δημιουργήθηκε.")

# Έλεγχος: Εμφάνιση των πρώτων γραμμών
print("Δείγμα υπολογισμού Sub_metering_4:")
display(df[['Global_active_power', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Sub_metering_4']].head())

print(df.head())

Το χαρακτηριστικό 'Sub_metering_4' δημιουργήθηκε.
Δείγμα υπολογισμού Sub_metering_4:


,Global_active_power,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4
datetime,,,,,
2006-12-16 17:24:00,4.216,0.0,1.0,17.0,52.266667
2006-12-16 17:25:00,5.360,0.0,1.0,16.0,72.333333
2006-12-16 17:26:00,5.374,0.0,2.0,17.0,70.566667
2006-12-16 17:27:00,5.388,0.0,1.0,17.0,71.800000
2006-12-16 17:28:00,3.666,0.0,1.0,17.0,43.100000


                     Global_active_power  Global_reactive_power  Voltage  \
datetime                                                                   
2006-12-16 17:24:00                4.216                  0.418   234.84   
2006-12-16 17:25:00                5.360                  0.436   233.63   
2006-12-16 17:26:00                5.374                  0.498   233.29   
2006-12-16 17:27:00                5.388                  0.502   233.74   
2006-12-16 17:28:00                3.666                  0.528   235.68   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
datetime                                                                
2006-12-16 17:24:00              18.4             0.0             1.0   
2006-12-16 17:25:00              23.0             0.0             1.0   
2006-12-16 17:26:00              23.0             0.0             2.0   
2006-12-16 17:27:00              23.0             0.0             1.0   
2006-12-16 17:28:00          

Επαναδειγματοληψία (Resampling)

In [7]:
df_daily = df.resample('D').agg({ # D = daily
    'Global_active_power': 'sum',
    'Global_reactive_power': 'mean',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum'
})

Δημιουργία Daily_total_power

In [8]:
df_daily.rename(columns={'Global_active_power': 'Daily_total_power'}, inplace=True)

Δημιουργία Peak_hour_power

In [9]:
hourly_consumption = df['Global_active_power'].resample('h').sum() # h = hourly
peak_hour_series = hourly_consumption.resample('D').max()

Δημιουργία Nighttime_usage

In [10]:
night_mask = (df.index.hour >= 22) | (df.index.hour < 6)
df_night = df[night_mask]
night_usage_series = df_night['Global_active_power'].resample('D').sum()

Προσθήκη χαρακτηριστικών ως columns

In [11]:
df_daily['Peak_hour_power'] = peak_hour_series
df_daily['Night_usage_power'] = night_usage_series

Κατηγορικά χαρακτηριστικά

In [12]:
df_daily['Day_of_week'] = df_daily.index.dayofweek
df_daily['Is_workday'] = df_daily['Day_of_week'].apply(lambda x: 1 if x < 5 else 0) # Δευτέρα - Παρασκευή
df_daily['Is_weekend'] = df_daily['Day_of_week'].apply(lambda x: 1 if x >= 5 else 0) # ΣΚ
df_daily['Month'] = df_daily.index.month

In [13]:
def get_season(month):
    if month in [12,1,2]: return 1 # Winter
    elif month in [3,4,5]: return 2 # Spring
    elif month in [6,7,8]: return 3 # Summer
    else: return 4 # Fall

df_daily['Season'] = df_daily['Month'].apply(get_season)

Δημιουργία Weekend_usage

In [14]:
df_daily['Weekend_usage'] = df_daily.apply(
    lambda row: row['Daily_total_power'] if row['Is_weekend'] == 1 else 0,
    axis=1
)

In [15]:
df_daily.fillna(0, inplace=True)

In [16]:
print(df_daily.head())

            Daily_total_power  Global_reactive_power     Voltage  \
datetime                                                           
2006-12-16           1209.176               0.088187  236.243763   
2006-12-17           3390.460               0.156949  240.087028   
2006-12-18           2203.826               0.112356  241.231694   
2006-12-19           1666.194               0.104821  241.999313   
2006-12-20           2225.748               0.111804  242.308062   

            Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  \
datetime                                                                       
2006-12-16         13.082828             0.0           546.0          4926.0   
2006-12-17          9.999028          2033.0          4187.0         13341.0   
2006-12-18          6.421667          1063.0          2621.0         14018.0   
2006-12-19          4.926389           839.0          7602.0          6197.0   
2006-12-20          6.467361             0.

Converting to CSV

In [ ]:
# --- Βήμα 4: Εξαγωγή Δεδομένων (Export) ---

# Αποθήκευση του τελικού ημερήσιου dataset σε αρχείο CSV
# Χρησιμοποιούμε index=True για να αποθηκευτεί και η στήλη 'datetime' (ημερομηνίες)
df_daily.to_csv('household_power_daily.csv', index=True)

print("Η διαδικασία ολοκληρώθηκε.")
print("Το αρχείο 'household_power_daily.csv' δημιουργήθηκε επιτυχώς και είναι έτοιμο για χρήση στο Notebook Μοντελοποίησης.")

# Προαιρετικός έλεγχος: Διαβάζουμε το αρχείο για να βεβαιωθούμε ότι είναι σωστό
df_check = pd.read_csv('household_power_daily.csv')
display(df_check.head())